### Задача 2. Функция вычисления метрики
 

Напишите функцию `get_metric`, которая вычисляет значения метрики из предыдущего задания.

 

Шаблон решения


```python
import pandas as pd


def get_metric(df, support_cost):
    """Вычисляет значения метрики.

    :param df (pd.DataFrame): таблица с данными о заказах со столбцами:
        - revenue - выручка
        - cost_price - себестоимость
        - support - обращение в поддержку
    :param support_cost (float): средняя стоимость одного обращения в поддержку
    :return (pd.Series): значения метрики
    """
    # YOUR_CODE_HERE
```

Пример

```python
df = pd.DataFrame({
    'revenue': [1500, 1800, 2100],
    'cost_price': [1300, 1200, 1600],
    'support': [1, 0, 0],
})
support_cost = 110
metric = get_metric(df, support_cost)
# metric = pd.Series([90, 600, 500])

In [1]:
import pandas as pd


def get_metric(df, support_cost):
    """Вычисляет значения метрики.

    :param df (pd.DataFrame): таблица с данными о заказах со столбцами:
        - revenue - выручка
        - cost_price - себестоимость
        - support - обращение в поддержку
    :param support_cost (float): средняя стоимость одного обращения в поддержку
    :return (pd.Series): значения метрики
    """
    
    return df['revenue'] - df['cost_price'] - df['support'] * support_cost

In [2]:
df = pd.DataFrame({
    'revenue': [1500, 1800, 2100],
    'cost_price': [1300, 1200, 1600],
    'support': [1, 0, 0],
})
support_cost = 110
metric = get_metric(df, support_cost)
metric
# metric = pd.Series([90, 600, 500])

0     90
1    600
2    500
dtype: int64

### Задача 3. Пороговое значение
 

Определите пороговое значении стоимости обращения в поддержку, при небольшом отклонение от которого меняется результат проверки гипотезы. Другими словами нужно найти значение, при котором p-value совпадает с уровнем значимости и среднее в экспериментальной группе больше, чем в контрольной.

Проверяем гипотезу о равенстве средних для метрики, которую вычисляли в предыдущем задании. В качестве критерия используем тест Стьюдента. Уровень значимости равен 0.05.

Ответ округлите до целого значения.

Код для генерации данных эксперимента

```python

import numpy as np
import pandas as pd

sample_size = 1000
df_control = pd.DataFrame({
    'revenue': [int(np.sin(x / 12) * 600 + 1200) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 12) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 400).astype(int),
})
df_pilot = pd.DataFrame({
    'revenue': [int(np.sin(x / 11 + 1) * 650 + 1250) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 11 + 1) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 300).astype(int),
})

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

sample_size = 1000
df_control = pd.DataFrame({
    'revenue': [int(np.sin(x / 12) * 600 + 1200) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 12) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 400).astype(int),
})
df_pilot = pd.DataFrame({
    'revenue': [int(np.sin(x / 11 + 1) * 650 + 1250) for x in range(sample_size)],
    'cost_price': [int(np.sin(x / 11 + 1) * 400 + 700) for x in range(sample_size)],
    'support': (np.arange(sample_size) < sample_size - 300).astype(int),
})

In [4]:
for i in range(0, 500):
    alpha = 0.05
    support_cost = i
    metric_control = get_metric(df_control, support_cost)
    metric_pilot = get_metric(df_pilot, support_cost)

    metric_control_mean, metric_pilot_mean = metric_control.mean(), metric_pilot.mean()

    pvalue = ttest_ind(metric_control, metric_pilot).pvalue

    if pvalue == alpha and metric_pilot_mean>metric_control_mean:
        print(support_cost)

In [10]:
alpha = 0.05
threshold_found = False

for support_cost in range(0, 500):
    metric_control = get_metric(df_control, support_cost)
    metric_pilot = get_metric(df_pilot, support_cost)

    pvalue = ttest_ind(metric_control, metric_pilot).pvalue
    diff = metric_pilot.mean() - metric_control.mean()

    if not threshold_found and abs(pvalue-alpha) < 0.001 and diff > 0:
        print(pvalue, alpha, diff)
        print(f"Пороговое значение support_cost: {support_cost}")
        threshold_found = True
        break



0.04992404291974633 0.05 19.38900000000001
Пороговое значение support_cost: 317
